In [ ]:
!pip install tsfresh==0.17.0
!pip install h5py
!pip install yfinance

     |████████████████████████████████| 91 kB 3.8 MB/s 
     |████████████████████████████████| 830 kB 40.2 MB/s 
     |████████████████████████████████| 133 kB 46.3 MB/s 
     |████████████████████████████████| 822 kB 46.4 MB/s 
     |████████████████████████████████| 802 kB 27.4 MB/s 
     |████████████████████████████████| 802 kB 47.5 MB/s 
     |████████████████████████████████| 793 kB 30.5 MB/s 
     |████████████████████████████████| 793 kB 45.3 MB/s 
     |████████████████████████████████| 791 kB 35.9 MB/s 
     |████████████████████████████████| 786 kB 44.1 MB/s 
     |████████████████████████████████| 779 kB 43.0 MB/s 
     |████████████████████████████████| 778 kB 12.7 MB/s 
     |████████████████████████████████| 776 kB 38.1 MB/s 
     |████████████████████████████████| 769 kB 46.7 MB/s 
     |████████████████████████████████| 766 kB 40.3 MB/s 
     |████████████████████████████████| 1.0 MB 28.9 MB/s 
     |████████████████████████████████| 722 kB 41.4 MB/s 
     |██████████

     |████████████████████████████████| 6.4 MB 3.9 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import os
import datetime
from random import sample
import statistics
from keras.models import model_from_json
import xgboost as xgb
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score

from tsfresh.feature_extraction import extract_features
from tsfresh.feature_selection import select_features
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import MinimalFCParameters

import pickle
import joblib as jb
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame
import seaborn as sns
import tensorflow as tf
import tsfresh as tfrs
import json
import pandas as pd
import yfinance as yf
import random

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

#Parametres
pred_len = 5
out_step = 30
MAX_EPOCHS = 500
history_lgt = 1000
patience_v = 25

#model metric
loss_metric = tf.losses.MeanSquaredError()
obj_metric = tf.metrics.MeanAbsoluteError()
#baseline metric
b_loss_metric = tf.losses.MeanSquaredError()
b_obj_metric = tf.metrics.MeanAbsoluteError()

#train_p > test_p leave some space for validation
train_p = 0.60
test_p = 0.30

#lstm params
n_layers = 5
n_units = 1
n_models_t = 8

# configure bootstrap
n_iterations = 10
boot_percent = 0.90

#configure metalearner
tolerance = 0.01

#XGBoost params
test_s = 0.10
xgb_estimator = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                  colsample_bynode=1, colsample_bytree=1, gamma=0,
                  learning_rate=0.1, max_delta_step=0, max_depth=15, n_iterations = 1000,
                  min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
                  nthread=None, objective='binary:logistic', eval_metric = 'error', random_state=0,
                  reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
                  silent=None, subsample=1, verbosity=1)

multi_val_performance = {}
val_performance = {}
multi_performance = {}
performance = {}
lstm_models = {}
models = {}
training_mat = {}
xgb_models = {}
sel_mat = {}

with open("/content/drive/MyDrive/Models/securities_dict_01_2022.json") as json_file:
  tsx_securities_dict = json.load(json_file)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
def get_current_price(symbol):
    ticker = yf.Ticker(symbol+'.TO')
    todays_data = ticker.history(period='1d')
    return todays_data['Close'][0]

def fetch_finance_data(hist_lgt, ticker_list):
  df_empty = pd.DataFrame()

  for keys in list(ticker_list.keys()):
    keys = keys.replace(".", "-")
    money_df = pd.DataFrame(yf.download(f'{keys}.TO',
                          progress=False))
    #duplicate index problems
    money_df = money_df[~money_df.index.duplicated(keep='first')]
    if(len(money_df["Close"])>=hist_lgt):
      df_empty[keys] = money_df["Close"].iloc[-hist_lgt:]
  df_empty['Date'] = df_empty.index
  return df_empty.dropna()

def partition (list_in, n):
    random.shuffle(list(list_in))
    return [list_in[i::n] for i in range(n)]

def assemble_matrix(training_mat):
    for j in range(0,len(training_mat)):
      if j==0:
        table_ass = training_mat[j].iloc[:,:-1]
      if j>0:
        table_ass = pd.concat([table_ass, training_mat[j].iloc[:,:-1]], axis=1, join='inner')
    for j in range(0,len(training_mat)):
        table_ass[training_mat[j].columns[-1]]= training_mat[j].iloc[:,-1]
    return table_ass

def train_xgb(num_label, training_m, test_size, xgb_estimator):
    X = training_m.iloc[:-1,:-num_label]
    y = training_m.iloc[1:,-num_label:]
    z = training_m.iloc[:,:-num_label]
    # split dataset into training and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=123, shuffle=False)
    # create XGBoost instance with default hyper-parameters

    # create MultiOutputClassifier instance with XGBoost model inside
    multilabel_model = MultiOutputClassifier(xgb_estimator)
    multilabel_model.fit(X_train, y_train)

    z_predict = multilabel_model.predict(z)

    return multilabel_model, z_predict, z

def gen_label(bs_table, sel_mat):
  df = sel_mat.iloc[-(bs_table.shape[0]+1):,-1].pct_change().dropna()
  df.index = bs_table.index
  df[df > 0] = 1 ; df[df <= 0] = 0
  return df

def reassemble_df_bs(models, loaded_df):
  predict_df = models.predict(loaded_df[0].test)[:,:,:]
  bite_lgt = predict_df.shape[1]
  num_bite = predict_df.shape[0]
  df = pd.DataFrame(np.concatenate(predict_df[:,:,:][0][0:(bite_lgt)]))
  for j in range(1,num_bite):
    df.loc[bite_lgt+j] = predict_df[:,:,:][j][(bite_lgt-1)]  # adding a row
  return df

def reassemble_full_bs(models, loaded_df):
  for i in models:
    if(i == 0):
      df = reassemble_df_bs(models[i], loaded_df)
    if(i > 0):
      df[str(i)] = reassemble_df_bs(models[i], loaded_df)
  return df

def build_bs_ts(df, loaded_df):
  df_lgt = df.shape[0] ; bs_lgt = df.shape[1]
  res = pd.DataFrame(df.iloc[0,:]) ; res.index = range(0,bs_lgt)
  for i in range(1,df_lgt):
    for j in range(0,bs_lgt):
      res.loc[i*bs_lgt + j] = df.iloc[i,j]
  res.columns = [loaded_df[1].columns[-1]]
  return res

def error_table(bs_t, loaded_df):
  rl_value = loaded_df[1].iloc[1:,-1]
  rl_value.index = bs_t.index
  for j in range(0,bs_t.shape[1]):
    bs_t.iloc[:,j] = abs(bs_t.iloc[:,j] - rl_value)
  return bs_t

def groupedAvg(myArray, N=2):
  result = myArray.groupby(myArray.index // N).sum()/N
  return result

def fetch_extract_features_df(ts_data, label_col_name, pred_len, custom_params = None, cl_label = None):
    rm = ts_data[label_col_name].size%pred_len
    len_df = ts_data[label_col_name].size
    if(rm!= 0):
      ts_data = ts_data[rm:len_df]

    df  = DataFrame(ts_data[label_col_name])
    df2 = DataFrame(np.repeat(range(int(len_df/pred_len)),pred_len))
    df3 = DataFrame(np.tile(range(pred_len),int(len_df/pred_len)))
    df2.index = df3.index = df.index
    df["ID"] = df2
    df["Time"] = df3
    if(custom_params is None):
      df_target = df[label_col_name][(pred_len-1)::pred_len]

    #feature computing and selecting
    if(custom_params is None):
      settings = ComprehensiveFCParameters()
    if(custom_params is not None):
      settings = custom_params
      #calculating predicted return change and rearranging index
      df_target = groupedAvg(df[label_col_name],N=pred_len)[1:]
      df_target2 = groupedAvg(df[label_col_name],N=pred_len)[:-1]
      df_target.index = df_target2.index; df_target3 = df_target - df_target2
      df_target3.loc[-1] = 0  # adding a row
      df_target3.index = df_target3.index + 1  # shifting index
      df_target3 = df_target3.sort_index()  # sorting by index
      df_target = df_target3

    extracted_features = extract_features(df, default_fc_parameters=settings, column_id="ID", column_sort="Time")


    df_target.index = extracted_features.index
    extracted_features = pd.DataFrame.dropna(extracted_features, axis = 1)
    if(custom_params is None):
      selected_features = select_features(extracted_features, df_target)
      #adding price for lstm target value
      selected_features[label_col_name] = df_target
    if(custom_params is not None):
      selected_features = extracted_features
      selected_features[label_col_name] = df_target

    return selected_features

def train_lstm(selected_features,pred_len,out_step,MAX_EPOCHS,patience_v,loss_metric,obj_metric,
               b_loss_metric,b_obj_metric,train_p,test_p,n_layers,n_units, skip_training):
    #data normalisation
    label_col_name = selected_features.columns[-1]
    column_indices = {name: i for i, name in enumerate(selected_features.columns)}
    n = len(selected_features)
    train_df = selected_features[0:int(n*train_p)]
    val_df = selected_features[int(n*train_p):int(n*(1-test_p))]
    test_df = selected_features[int(n*(1-test_p)):]
    predict_df = selected_features[-(out_step+1):]
    predict_df.append([predict_df.iloc[-1:,:]],ignore_index=True)

    num_features = selected_features.shape[1]

    train_mean = train_df.mean()
    train_std = train_df.std()

    train_df = (train_df - train_mean) / train_std
    val_df = (val_df - train_mean) / train_std
    test_df = (test_df - train_mean) / train_std
    predict_df = (predict_df - train_mean) / train_std

    #TensorFlow window generator
    class WindowGenerator():
      def __init__(self, input_width, label_width, shift,
                  train_df=train_df, val_df=val_df, test_df=test_df, predict_df=predict_df,
                  label_columns=None):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df
        self.predict_df = predict_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
          self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                              enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]
        self.predict_slice = slice(0, self.total_window_size)
        self.predict_indices = np.arange(self.total_window_size)[self.predict_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

      def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])

    def split_window(self, features):
      inputs = features[:, self.input_slice, :]
      labels = features[:, self.labels_slice, :]
      if self.label_columns is not None:
        labels = tf.stack(
            [labels[:, :, self.column_indices[name]] for name in self.label_columns],
            axis=-1)
      # Slicing doesn't preserve static shape information, so set the shapes
      # manually. This way the `tf.data.Datasets` are easier to inspect.
      inputs.set_shape([None, self.input_width, None])
      labels.set_shape([None, self.label_width, None])

      return inputs, labels


    WindowGenerator.split_window = split_window

    def plot(self, model=None, plot_col=label_col_name, max_subplots=3):
      inputs, labels = self.example
      plt.figure(figsize=(12, 8))
      plot_col_index = self.column_indices[plot_col]
      max_n = min(max_subplots, len(inputs))
      for n in range(max_n):
        plt.subplot(3, 1, n+1)
        plt.ylabel(f'{plot_col} [normed]')
        plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                label='Inputs', marker='.', zorder=-10)

        if self.label_columns:
          label_col_index = self.label_columns_indices.get(plot_col, None)
        else:
          label_col_index = plot_col_index

        if label_col_index is None:
          continue

        plt.scatter(self.label_indices, labels[n, :, label_col_index],
                    edgecolors='k', label='Labels', c='#2ca02c', s=64)
        if model is not None:
          predictions = model(inputs)
          plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                      marker='X', edgecolors='k', label='Predictions',
                      c='#ff7f0e', s=64)

        if n == 0:
          plt.legend()

      plt.xlabel('Time [h]')

    WindowGenerator.plot = plot

    def make_dataset(self, data):
      data = np.array(data, dtype=np.float32)
      ds = tf.keras.preprocessing.timeseries_dataset_from_array(
          data=data,
          targets=None,
          sequence_length=self.total_window_size,
          sequence_stride=1,
          shuffle=False,
          batch_size=32,)

      ds = ds.map(self.split_window)

      return ds

    WindowGenerator.make_dataset = make_dataset

    @property
    def train(self):
      return self.make_dataset(self.train_df)
    @property
    def val(self):
      return self.make_dataset(self.val_df)
    @property
    def test(self):
      return self.make_dataset(self.test_df)
    @property
    def predict(self):
      return self.make_dataset(self.predict_df)
    @property
    def example(self):
      """Get and cache an example batch of `inputs, labels` for plotting."""
      result = getattr(self, '_example', None)
      if result is None:
        # No example batch was found, so get one from the `.train` dataset
        result = next(iter(self.train))
        # And cache it for next time
        self._example = result
      return result

    WindowGenerator.train = train
    WindowGenerator.val = val
    WindowGenerator.test = test
    WindowGenerator.predict = predict
    WindowGenerator.example = example

    def compile_and_fit(model, window, patience=2):
      early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                                        patience=patience_v,
                                                        mode='min')

      model.compile(loss=loss_metric,
                    optimizer=tf.optimizers.Adam(),
                    metrics=[obj_metric])

      history = model.fit(window.train, epochs=MAX_EPOCHS,
                          validation_data=window.val,
                          callbacks=[early_stopping])
      return history

    class Baseline(tf.keras.Model):
      def __init__(self, label_index=None):
        super().__init__()
        self.label_index = label_index

      def call(self, inputs):
        if self.label_index is None:
          return inputs
        result = inputs[:, :, self.label_index]
        return result[:, :, tf.newaxis]

    baseline = Baseline(label_index=column_indices[label_col_name])

    baseline.compile(loss=b_loss_metric,
                    metrics=[b_obj_metric])

    #Window generation
    wide_window = WindowGenerator(
        input_width=out_step, label_width=out_step, shift=1,
        label_columns=[label_col_name])
    if skip_training == False:
      #lstm training and output
      lstm_model = tf.keras.models.Sequential([
          # Shape [batch, time, features] => [batch, time, lstm_units]
          tf.keras.layers.LSTM(n_layers, return_sequences=True),
          # Shape => [batch, time, features]
          tf.keras.layers.Dense(units=n_units)
      ])
      history = compile_and_fit(lstm_model, wide_window)
      return lstm_model, wide_window, test_df, predict_df
    if skip_training == True:
      return wide_window, test_df, predict_df

**Training LSTM**

In [ ]:
for j in range(3,4):
  #only selecting our desired history length
  ts_data = fetch_finance_data(history_lgt,tsx_securities_dict)
  label_col_name = ts_data.columns.values[j]
  #from the time series we extract relevant features using tsfresh, outputs lstm training matrix
  selected_features = fetch_extract_features_df(ts_data, label_col_name, pred_len)
  #saving training matrix with pickle
  with open("/content/drive/MyDrive/Models_df/"+str(label_col_name)+".pickle", 'wb') as handle:
    pickle.dump(selected_features, handle, protocol=pickle.HIGHEST_PROTOCOL)
  #fetching the numbers of column to slect for bootstrap
  n_size = int((len(selected_features.columns)-1)*boot_percent)
  lstm_models = {}
  for i in range(0,n_iterations):
    #randomly selecting a percentage of the columns (features) for bootstrapping procedure
    boot = sample(range(0,(len(selected_features.columns)-2)), n_size)
    #storing bootstrap table in variable
    selected_features2 = selected_features.iloc[:,boot]
    selected_features2[label_col_name] = selected_features[label_col_name]
    #training lstm model and fetching window object and partial training matrix for testing and predicting
    lstm_model, wide_window, test_df, predict_df = train_lstm(selected_features,pred_len,out_step,MAX_EPOCHS,patience_v,loss_metric,obj_metric,
                  b_loss_metric,b_obj_metric,train_p,test_p,n_layers,n_units, skip_training = False)
    model_json = lstm_model.to_json()
    # serialize model to JSON
    with open("/content/drive/MyDrive/Models/"+str(label_col_name)+str(i)+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    lstm_model.save_weights("/content/drive/MyDrive/Models/"+str(label_col_name)+str(i)+".h5")
    print("Saved model to disk")

**Loading models**

In [ ]:
loaded_models = {}; loaded_df = {}
for j in range(0,4):
  ts_data = fetch_finance_data(history_lgt,tsx_securities_dict)
  label_col_name = ts_data.columns.values[j]
  loaded_model = {}
  for i in range(0,n_iterations):
    # load json and create model
    json_file = open("/content/drive/MyDrive/Models/"+str(label_col_name)+str(i)+".json", 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model[i] = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model[i].load_weights("/content/drive/MyDrive/Models/"+str(label_col_name)+str(i)+".h5")
    # recompile model using precalculated weights
    loaded_model[i].compile(loss=loss_metric, optimizer=tf.optimizers.Adam(), metrics=[obj_metric])
    print("Loaded model from disk")
  loaded_models[j] = loaded_model
  #fetching lstm training matrix
  with open("/content/drive/MyDrive/Models_df/"+str(label_col_name)+".pickle", 'rb') as handle:
    selected_feature = pickle.load(handle)
  #only fetching window object and partial training matrix, skipping lstm training
  loaded_df[j] = train_lstm(selected_feature,pred_len,out_step,MAX_EPOCHS,patience_v,loss_metric,obj_metric,
                b_loss_metric,b_obj_metric,train_p,test_p,n_layers,n_units, skip_training = True)
  sel_mat[j] = selected_feature

for i in range(0,4):
  # assembling every test prediction of the bootstrap into one fat dataframe
  bs_table = reassemble_full_bs(loaded_models[i], loaded_df[i])
  # calculating real errors from this fat dataframe
  er_tabble = error_table(bs_table,loaded_df[i])
  # putting this dataframe as one big time series by transposing every row and contacatenating them all
  bs_ts = build_bs_ts(er_tabble, loaded_df[i])
  # using this big timeseries to extract features from the variation of errors from the bootstrap at every timestep (xgboost training matrix)
  bs_features = fetch_extract_features_df(bs_ts, loaded_df[i][1].columns[-1], n_iterations, custom_params=MinimalFCParameters())
  # generating -1 and 1 label if the stock goes up or down
  bs_features[loaded_df[i][1].columns[-1]+"_label"] = gen_label(bs_features, sel_mat[i])
  # storing the training matrix
  training_mat[i] = bs_features
  with open("/content/drive/MyDrive/Models_df/bs_ts"+str(i)+".pickle", 'wb') as handle:
    pickle.dump(training_mat[i], handle, protocol=pickle.HIGHEST_PROTOCOL)

Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk


Feature Extraction: 100%|██████████| 5/5 [00:00<00:00, 116.35it/s]


In [ ]:
training_mat = {}
with open("/content/drive/MyDrive/Models/securities_dict_01_2022.json") as json_file:
  tsx_securities_dict = json.load(json_file)
for j in range(0,4):
  with open("/content/drive/MyDrive/Models_df/bs_ts"+str(j)+".pickle", 'rb') as handle:
    training_mat[j] = pickle.load(handle)

ass_mat = assemble_matrix(training_mat)
xgb_model = train_xgb(4, ass_mat.iloc[1:,:], 0.2, xgb_estimator)
next_prediction = xgb_model[1][-1]

In [ ]:
def return_last_1(n):
  return n[-1][1]

x = map(return_last_1, xgb_model[0].predict_proba(xgb_model[2]))

In [ ]:

result_array = np.array(list(x))
print(list(tsx_securities_dict.keys())[0:4])
print(result_array/sum(result_array))

['AC', 'SHOP', 'SU', 'TD']
[0.13389073 0.34020227 0.30489552 0.22101142]


In [ ]:
feat_impts = []
for clf in xgb_model[0].estimators_:
    feat_impts.append(clf.feature_importances_)

print(np.mean(feat_impts, axis=0))
print(ass_mat.columns[:-4])

[0.02972176 0.03009943 0.         0.         0.04984279 0.
 0.03352922 0.03613449 0.02772861 0.02753341 0.04013626 0.
 0.         0.0540335  0.         0.03379419 0.01681332 0.03150366
 0.04594743 0.07191212 0.         0.         0.08945449 0.
 0.05454541 0.03914146 0.05595576 0.03742428 0.02333892 0.
 0.         0.03555537 0.         0.03651338 0.02712697 0.07221375]
Index(['AC__sum_values', 'AC__median', 'AC__mean', 'AC__length',
       'AC__standard_deviation', 'AC__variance', 'AC__maximum', 'AC__minimum',
       'AC', 'SHOP__sum_values', 'SHOP__median', 'SHOP__mean', 'SHOP__length',
       'SHOP__standard_deviation', 'SHOP__variance', 'SHOP__maximum',
       'SHOP__minimum', 'SHOP', 'SU__sum_values', 'SU__median', 'SU__mean',
       'SU__length', 'SU__standard_deviation', 'SU__variance', 'SU__maximum',
       'SU__minimum', 'SU', 'TD__sum_values', 'TD__median', 'TD__mean',
       'TD__length', 'TD__standard_deviation', 'TD__variance', 'TD__maximum',
       'TD__minimum', 'TD'],
   

In [ ]:
#Deprecated functions

#ts_data, ts_metadata = pull_daily_time_series_alpha_vantage(alpha_vantage_api_key, ticker_name = ticker_n, output_size = "full")
from alpha_vantage.timeseries import TimeSeries

alpha_vantage_api_key = "HTNGJULEJU3C38H5"

def pull_daily_time_series_alpha_vantage(alpha_vantage_api_key, ticker_name, output_size = "compact"):
    """
    Pull daily time series by stock ticker name.
    Args:
        alpha_vantage_api_key: Str. Alpha Vantage API key.
        ticker_name: Str. Ticker name that we want to pull.
        output_size: Str. Can be "full" or "compact". If "compact", then the past 100 days of data
        is returned. If "full" the complete time series is returned (could be 20 years' worth of data!)
    Outputs:
        data: Dataframe. Time series data, including open, high, low, close, and datetime values.
        metadata: Dataframe. Metadata associated with the time series.
    """
    #Generate Alpha Vantage time series object
    ts = TimeSeries(key = alpha_vantage_api_key, output_format = 'pandas')
    data, meta_data = ts.get_daily_adjusted(ticker_name, outputsize = output_size)
    data['date_time'] = data.index
    data = data[::-1]
    return data, meta_data